## sub-task 4 제출 스크립트
ETRI FASHION-HOW Season 5 task4 제출코드입니다.
#### 주의: 반드시 본 파일을 이용하여 제출을 수행해야 하며 파일의 이름은 task.ipynb로 유지되어야 합니다.
- 작성하신 추론용 코드(예: main.py)를 본 스크립트 내의 etri_task4_submit() 함수로 작동되게끔 삽입하는 것으로 결과 제출을 수행할 수 있습니다.
- Zero-shot learning을 평가하는 과제로, 학습한 모델을 업로드하여 학습한 적이 없는 데이터에 대해 추론을 수행하도록 구성합니다.


코드는 크게 4가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하시면 되겠습니다.
1. 제출용 aifactory 라이브러리 설치
2. etri_task4_submit() 함수 편집 (추론 스크립트)
3. submit() 함수로 wrapping
4. if __name__ == "__main__" 조건문을 이용한 제출 수행

※ 가능하면 제출시에는 포함되어 있는 train data를 폴더에서 제외하고 제출하시는 편이 좋습니다.
- 파일 크기 감소 → 업로드 시간 감소 → 전체 추론 수행 시간 감소

### 1. 제출용 aifactory 라이브러리 설치
#### 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

In [ ]:
!pip install -U aifactory

### 2. etri_task4_submit() 함수 편집 (추론 스크립트)
#### etri_task4_submit() 편집 시 주의사항

1. 아래 etri_task4_submit() 함수 내에 전체 추론 실행 코드를 삽입, 테스트셋에 대하여 추론을 수행하고 결과를 list로 return하게끔 구성
   - Baseline이 아닌 다른 모델을 사용하는 경우에도 동일
2. 함수 내에서는 import * 가 적용되지 않으므로 필요한 import object를 직접 입력
   - 반드시 함수 내에서 import가 이루어져야 합니다.
3. argparse 사용시 args, _ = parser.parse_known_args()로 인자 지정
   args = parser.parse_args()는 jupyter에서 오류가 발생합니다!!!
4. 모델 내부의 경로는 ./ 으로 경로를 지정합니다. (예: weight 파일 경로 = ./model/...)
5. 데이터는 **/aif/data/** 경로 아래에 있습니다. (코드 내용 참조)
   - subWordEmb_path는 /aif/sstm_v0p5_deploy/ 안에 .dat 파일이 있습니다.
6. return할 결과물과 양식에 유의합니다.

In [ ]:
# main.py의 내용을 test 옵션과 제출 후 자동추론 환경에 맞게끔 수정하여 함수화

def etri_task4_submit():
    
    import argparse
    import torch
    from gaia import FahsionHowDataset, Model, gAIa

    def get_udevice():
        """
        function: get usable devices(CPU and GPU)
        """
        if torch.cuda.is_available():
            device = torch.device('cuda')
            num_gpu = torch.cuda.device_count()
        else:    
            device = torch.device('cpu')
        print('Using device: {}'.format(device))
        if torch.cuda.is_available():
            print('# of GPU: {}'.format(num_gpu))
        return device


    def str2bool(v):
        """
        function: convert into bool type(True or False)
        """
        if isinstance(v, bool): 
            return v 
        if v.lower() in ('yes', 'true', 't', 'y', '1'): 
            return True 
        elif v.lower() in ('no', 'false', 'f', 'n', '0'): 
            return False 
        else: 
            raise argparse.ArgumentTypeError('Boolean value expected.')
            
    # input options
    parser = argparse.ArgumentParser(description='AI Fashion Coordinator.')

    parser.add_argument('--mode', type=str, 
                        default='zsl', # 'zsl'로 추론 수행
                        help='training or test or zero-shot mode')
    parser.add_argument('--in_file_trn_dialog', type=str, 
                        default='/aif/data/ddata.wst.txt.2023.08.29', # 경로 주의
                        help='training dialog DB')
    parser.add_argument('--in_file_tst_dialog', type=str, 
                        default='/aif/data/fs_eval_t1.wst.tst.2024', # 경로 및 파일명에 유의
                        help='test dialog DB')
    parser.add_argument('--in_file_fashion', type=str, 
                        default='/aif/data/mdata.wst.txt.2023.08.23', # 경로 주의
                        help='fashion item metadata')
    parser.add_argument('--in_dir_img_feats', type=str, 
                        default='/aif/data/img_feats', # 경로 주의
                        help='fashion item image features')
    parser.add_argument('--model_path', type=str, 
                        default='./model', 
                        help='path to save/read model')
    parser.add_argument('--model_file', type=str, 
                        default='gAIa-500.pt',
                        help='model file name')
    parser.add_argument('--subWordEmb_path', type=str, 
                        default='/aif/sstm_v0p5_deploy/sstm_v4p49_np_n36134_d128.dat', # 경로 주의
                        help='path of subword embedding')
    parser.add_argument('--req_net_type', type=str,
                        default='memn2n',
                        help='memory network or transformer')
    parser.add_argument('--eval_net_type', type=str,
                        default='tf', 
                        help='transformer or mlp')
    parser.add_argument('--eval_node', type=str, 
                        default='[600,4000,4000]', 
                        help='nodes of evaluation network')
    parser.add_argument('--learning_rate', type=float,
                        default=0.0001, 
                        help='learning rate')
    parser.add_argument('--max_grad_norm', type=float,
                        default=40.0, 
                        help='clip gradients to this norm')
    parser.add_argument('--eval_zero_prob', type=float,
                        default=0.0, 
                        help='dropout prob.')
    parser.add_argument('--tf_dropout', type=float,
                        default=0.3,   
                        help='dropout for nn.TransformerEncoderLayer')
    parser.add_argument('--corr_thres', type=float,
                        default=0.7, 
                        help='correlation threshold')
    parser.add_argument('--batch_size', type=int,
                        default=100,   
                        help='batch size for training')
    parser.add_argument('--epochs', type=int,
                        default=10,   
                        help='epochs to training')
    parser.add_argument('--save_freq', type=int,
                        default=2,   
                        help='evaluate and save results per # epochs')
    parser.add_argument('--hops', type=int,
                        default=3,   
                        help='number of hops in the MemN2N')
    parser.add_argument('--mem_size', type=int,
                        default=32,
                        help='memory size for the MemN2N')
    parser.add_argument('--key_size', type=int,
                        default=300,   
                        help='memory size for the MemN2N')
    parser.add_argument('--tf_nhead', type=int,
                        default=4,   
                        help='nhead for nn.TransformerEncoderLayer')
    parser.add_argument('--tf_ff_dim', type=int,
                        default=4096,   
                        help='dim_feedforward for nn.TransformerEncoderLayer')
    parser.add_argument('--tf_num_layers', type=int,
                        default=4,
                        help='num_layers for nn.TransformerEncoder')
    parser.add_argument('--evaluation_iteration', type=int,
                        default=100,   
                        help='# of test iteration')
    parser.add_argument('--use_batch_norm', type=str2bool, 
                        default=False, 
                        help='use batch normalization')
    parser.add_argument('--use_dropout', type=str2bool, 
                        default=False, 
                        help='use dropout')
    parser.add_argument('--use_multimodal', type=str2bool,
                        default=True, 
                        help='use multimodal input')
    parser.add_argument('--use_input_mask', type=str2bool, 
                        default=False, 
                        help='use input masking')

    args, _ = parser.parse_known_args()
    gaia = gAIa(args, get_udevice())
    out = gaia.zsl()
    return out

### 3. submit() 함수로 wrapping
#### 반드시 아래와 같이 submit() 이라는 함수로 위에 정의된 etri_task4_submit 함수를 wrapping해야 합니다.

In [ ]:
def submit():
    return etri_task4_submit

### 4. if name == "main" 조건문을 이용한 제출 수행
#### 아래와 같이 if __name__ == "__main__" 구문 내에서 제출 함수가 실행되게끔 합니다.
#### ※ task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 각팀 팀장분에게 메일로 할당된 계정을 이용하여 task별로 확인하실 수 있습니다.

In [ ]:
import aifactory.score as aif
import time
t = time.time()
if __name__ == "__main__":
    #-----------------------------------------------------#
    aif.submit(model_name="etri-task4_test",
               key="",
               func=submit # submit function
               )
    #-----------------------------------------------------#
    print(time.time() - t)